In [47]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from scipy.special import rel_entr
import random #GGC added library

# Fixed parameters

In [48]:
#number of parameter vectors
#samplesize = 5000
samplesize = 5000

#number of bins applied to build the histogram in the calculation of expressibility
nbins = 75

#width of the bins, considering that the fidelity values goes from 0 to 1
width = 1.0/nbins 

# Variable parameters (qubits, layers and distributions)

In [49]:
#List of distributions
#distributions = ['normal', 'uniform', 'exponential']
distributions = ['normal']

#List of number of qubits
#qubits = [4, 5, 6]
qubits = [8]

#List of number of layers
#layers = [1, 2, 3, 4, 5]
layers = [500]

# Multiple layer fidelity histogram

In [50]:
def fidel_histogram(dist,binsnumb, samples, circuit, nlayers):
    #store the realizations of the fidelities
    #0th_A Modif added by GGC, vector auxs added (is there any more elegant method in python?)

    fidel_vector = []
    CumAvg = []
    DCum = []
    
    #selects the distribution
    if dist == 'normal':

        #0th_B Modif added by GGC, sets to zero auxiliars CumAvg, Dcum (is there any more elegant method in python?)
        CumAvg=0
        DCum=0
        
        for i in range(samples):
            
            #this part generates the vectors of angles for the circuits. This is conditioned on the number of layers
            #and qubits. The array of parameters will have two angles (one for RX and one for RY) per qubit and
            #one vector of this kind for every layer. So, for the calculation of the expressibility, we sample
            #two of these parameter vectors and then apply then into a circuit.
            
            
            angles1 = np.array([ [[np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)], 
                                [np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            angles2 = np.array([ [[np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)], 
                                [np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            #Here the parameter vectors are applied to generate two states considering a particular circuit
            #structure that is given as an input for the function.
            
            state1 = circuit(angles1)
            state2 = circuit(angles2)

            #Circuit drawing

            #if circuit == random_circuit :
            #    params = [0.1, 0.2,.3,.4,.5,.6,.7,.8,.9,1.]
            #    drawer = qml.draw(circuit, show_all_wires=True, wire_order=[0,1,2,3])
            #    print(drawer(params))
            
            #Fidelity computation
            
            F = np.abs( np.dot(state1, state2) )**2
        
            fidel_vector.append(F)

            #1st Section added by GGC, SD of Lorenz curves, in samples loop, using final vector/each circuit set

            P = np.abs( state1 )**2

            #Decreasing ordering of Pcomp
            P[::-1].sort()
	
            #Cummulants calculation
            Cum=np.cumsum(P)

            #Cummulants average and squared average calculation
            CumAvg=CumAvg+Cum
            DCum=DCum+Cum**2

            #End of 1st Section added by GGC, SD of Lorenz curves

        
        
        fidel_vector = np.asarray(fidel_vector)
        # is this ok?
        # CumAvg = np.asarray(CumAvg)
        # DCum = np.asarray(DCum)

        binsize = 1.0/binsnumb
        bins = np.arange(0, 1. + binsize, binsize)
        
        #Generation of the histograms to compute D_{KL}
        
        histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]


        
        #2nd Section added by GGC, SD of Lorenz curves final computation at the end of samples loop

        SDL=np.sqrt(DCum/samples-(CumAvg/samples)**2)

        #2nd Section added by GGC, SD of Lorenz curves final computation at the end of samples loop


    
    #The same is done for the other distributions
    if dist == 'exponential':
        
        for i in range(samples):
        
            angles1 = np.array([ [[np.random.exponential(scale = 0.865) for i in range(wires)], 
                                [np.random.exponential(scale = 0.865) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            angles2 = np.array([ [[np.random.exponential(scale = 0.865) for i in range(wires)], 
                                [np.random.exponential(scale = 0.865) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            state1 = circuit(angles1)
            state2 = circuit(angles2)
        
            F = np.abs( np.dot(state1, state2) )**2
        
            fidel_vector.append(F)
        
        fidel_vector = np.asarray(fidel_vector)
    
        binsize = 1.0/binsnumb
        bins = np.arange(0, 1. + binsize, binsize)
    
        histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]    
        
        
    if dist == 'uniform':
    
        for i in range(samples):
        
            angles1 = np.array([ [[np.random.uniform(low=0, high=2*np.pi) for i in range(wires)], 
                                [np.random.uniform(low=0, high=2*np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            angles2 = np.array([ [[np.random.uniform(low=0, high=2*np.pi) for i in range(wires)], 
                                [np.random.uniform(low=0, high=2*np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            state1 = circuit(angles1)
            state2 = circuit(angles2)
            
            F = np.abs( np.dot(state1, state2) )**2
        
            fidel_vector.append(F)
        
        fidel_vector = np.asarray(fidel_vector)
    
        binsize = 1.0/binsnumb
        bins = np.arange(0, 1. + binsize, binsize)
    
        histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]

        #3rd modif. added by GGC, SD of Lorenz curves returned by fidel_histogram to use the same output file
    
    return histogram, bins, SDL

# Code that executes the different conditions

In [51]:
#In this part of the code, the circuits are executed with different distributions, number of circuit concatenations
#(layers) and different number of qubits. For each set of parameters, the data is stored in a .txt file

i1 = [] #GGC not sure about the need of this defintion


now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")
print("Tempo inicial: ")
print(now0)
print()

for dist in distributions:
    
    for wires in qubits:
        #This command initializes the quantum circuit of qubits in pennylane, with number of qubits given by wires
        #and considering analytical computations (shots=None), if we want to compute mean values of observables.
        #shots different from None would lead to a stochastic calculation of mean values.
        
        dev = qml.device("default.qubit", wires=wires, shots=None)
        
        
        for nlayers in layers:

            #GGC addition of random G3 circuit (I have left the input parameter (though useless) in order to keep code shape

            def layer_random(rots):
                i1 = random.sample(range(0,wires), 2)

                i2 = random.randint(1,3)
                if   i2 == 1 :
                     qml.Hadamard(wires=[i1[0]])
                elif i2 == 2 :
                     #qml.RZ(np.pi/4, wires=[i1[0]])
                     qml.T(wires=[i1[0]])
                else : 
                     qml.CNOT(wires=[i1[0],i1[1]])
        
            def randomC(rotations):
                qml.layer(layer_random, nlayers,rotations)
                return qml.state()

            random_circuit = qml.QNode(randomC, dev)

            #End of GGC addition of random G3 circuit

            
            #No connections
            
            #To define a circuit with different number of layers in pennylane, we need to define which is
            #the structure of each layer (layer_noconnec function below) and then define how it is concatenated.
            #As we are just stacking the layers, changing only the values of the parameters, we just have to tell
            #that we want it to make a layered circuit (noconnec) with the structure of layer_noconnec.
            #The return qml.state() means that the output of the circuit is the complete quantum state generated
            #in the form of a vector.
            
            def layer_noconnec(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
        
            def noconnec(rotations):
                qml.layer(layer_noconnec, nlayers, rotations)
                return qml.state()

            
            #In this part we are combining the defined circuit with the defined device, a.k.a. the qubits quantum
            #circuit.
            noconnec_circuit = qml.QNode(noconnec, dev)
        
        
            #The same is done with the other circuits, but now we add the CNOTs step. Each one of them will have a
            #different command to generate the couplings as we want.
            
            
            #Linear
            def layer_linear(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")

            def linear(rotations):
                qml.layer(layer_linear, nlayers, rotations)
                return qml.state()

            linear_circuit = qml.QNode(linear, dev)
        
        
        
            #Ring
            def layer_ring(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
                qml.CNOT(wires=[wires-1,0])
    
            def ring(rotations):
                qml.layer(layer_ring, nlayers, rotations)
                return qml.state()

            ring_circuit = qml.QNode(ring, dev)
        
        
        
            #Star
            def layer_star(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                for i in range(wires-1):
                    qml.CNOT(wires = [0, i+1])

            def star(rotations):
                qml.layer(layer_star, nlayers, rotations)
                return qml.state()

            star_circuit = qml.QNode(star, dev)

        #4th modif. added by GGC, SD of Lorenz curves and G3 RC recovered from fidel_histogram to use the same output file

            random_hist, bins, SDL_random = fidel_histogram(dist, nbins, samplesize, random_circuit, nlayers)

            #noconnec_hist, bins, SDL_noconnec = fidel_histogram(dist, nbins, samplesize, noconnec_circuit, nlayers)
            #linear_hist, bins, SDL_linear = fidel_histogram(dist, nbins, samplesize, linear_circuit, nlayers)
            #ring_hist, bins, SDL_ring = fidel_histogram(dist, nbins, samplesize, ring_circuit, nlayers)
            #star_hist, bins, SDL_star = fidel_histogram(dist, nbins, samplesize, star_circuit, nlayers)


            #Haar histogram for 'wires' qubits
            N = 2**wires #dimension of the Hilbert space
            histogram_Haar = []
            for i in range(nbins):
                histogram_Haar.append(  (1-bins[i])**(N-1) - (1-bins[i+1])**(N-1)  )

            #GGC Random G3 circuit addition
            
            D_KL_random = sum( rel_entr(random_hist, histogram_Haar) )

            #D_KL_noconnec = sum( rel_entr(noconnec_hist, histogram_Haar) )
            #D_KL_linear = sum( rel_entr(linear_hist, histogram_Haar) )
            #D_KL_ring = sum( rel_entr(ring_hist, histogram_Haar) )
            #D_KL_star = sum( rel_entr(star_hist, histogram_Haar) )


            f = open("expressibility_ansatz1_dist{0}_qubits{1}_layers{2}_samples{3}.txt"
                     .format(dist,wires,nlayers,samplesize), "w")

            print('Circuit,Expressibility', file = f)

            #GGC Random G3 circuit addition
            
            print('Random Circuit G3,', D_KL_random, file = f)

            #print('No connections,', D_KL_noconnec, file = f)
            #print('Linear,', D_KL_linear, file = f)
            #print('Ring,', D_KL_ring, file = f)
            #print('Star,', D_KL_star, file = f)

        #5th modif. added by GGC, SD and G3 circuit of Lorenz curves output

            f2 = open("SDL_randomG3_dist{0}_qubits{1}_layers{2}_samples{3}.dat"
                     .format(dist,wires,nlayers,samplesize), "w")
            for i in range(SDL_random.size):
                        print('{0:.4f} {1:.8f}'.format(i/SDL_random.size, SDL_random[i]), file = f2)
            #f2 = open("SDL_noconnections_dist{0}_qubits{1}_layers{2}_samples{3}.dat"
            #         .format(dist,wires,nlayers,samplesize), "w")
            #for i in range(SDL_noconnec.size):
            #            print('{0:.4f} {1:.8f}'.format(i/SDL_noconnec.size, SDL_noconnec[i]), file = f2)
            #f2 = open("SDL_linear_dist{0}_qubits{1}_layers{2}_samples{3}.dat"
            #         .format(dist,wires,nlayers,samplesize), "w")
            #for i in range(SDL_linear.size):
            #            print('{0:.4f} {1:.8f}'.format(i/SDL_linear.size, SDL_linear[i]), file = f2)
            #f2 = open("SDL_ring_dist{0}_qubits{1}_layers{2}_samples{3}.dat"
            #         .format(dist,wires,nlayers,samplesize), "w")            
            #for i in range(SDL_ring.size):
            #            print('{0:.4f} {1:.8f}'.format(i/SDL_ring.size, SDL_ring[i]), file = f2)
            #f2 = open("SDL_star_dist{0}_qubits{1}_layers{2}_samples{3}.dat"
            #         .format(dist,wires,nlayers,samplesize), "w")                
            #for i in range(SDL_star.size):
            #            print('{0:.4f} {1:.8f}'.format(i/SDL_star.size, SDL_star[i]), file = f2)
            
            print('Execution: distribution=',dist,'; qubits=',wires,'; layers=', nlayers)
            now1 = dt.datetime.now() #monitorar tempo de início
            now1 = now1.strftime("%Y-%m-%d %H:%M:%S")
            print("Tempo: ",now1)
            print()
     
    
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

f.close()

print()
print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Tempo inicial: 
2024-02-14 14:07:15

Execution: distribution= normal ; qubits= 8 ; layers= 500
Tempo:  2024-02-14 14:22:29


Tempo inicial: 
2024-02-14 14:07:15
Tempo final:
2024-02-14 14:22:29
